# Segmenting and Clustering

In [4]:
# install the beautiful soup package
!conda install bs4 --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |           py36_0         172 KB
    bs4-4.9.1                  |                0           4 KB
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.2         |           py36_0         157 KB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    soupsieve-2.0.1            |             py_0          33 KB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     pkgs/main/linux-64::beautifulsoup4-4.9.1-py36_0
  bs4                pkgs/main/noarch::bs4-4.9.1-0
  soupsieve          pkg

In [5]:
import random 
import numpy as np 
import pandas as pd
import requests # labarary for url request

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import bs4 as bs
import urllib.request

print('Libraries imported.')

Libraries imported.


## Method One
use the crawler to scrape the following Wikipedia page
https://en.wikipedia.org

In [6]:
# Method One use the crawler to scrape the following Wikipedia page, https://en.wikipedia.org
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'html.parser') # return xml file

table = soup.find('table') # locate table
# print(table)
table_rows = table.find_all('tr') # locate all item  

# <td>M1A
# </td>
# <td>Not assigned
# </td>
# <td>Not assigned
# </td></tr>, <tr>

# print(table_rows)

l = []
for lines in table_rows:
    td = lines.find_all('td')
    row = [line.text.strip() for line in td if line.text.strip()]
    if row:
        l.append(row)
     

# change the data type from list to dataframe
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


## Method Two

In [26]:
# !conda install lxml --yes
# after install need to reload

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-4.5.1                 |   py36hefd8a0e_0         1.2 MB
    ------------------------------------------------------------
                                           Total:         1.6 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               pkgs/main/linux-64::lxml-4.5.1-py36hefd8a0e_0



lxml-4.5.1           | 1.2 MB    | ##################################### | 100% 
libxslt-1.1.33       | 426 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
# url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# data, = pd.read_html(url, match="Postal code", skiprows=1)
# data.columns = ["PostalCode", "Borough", "Neighborhood"]
# data.head()

In [8]:
# drop hte not assigned value
df = df[df.Borough!='Not assigned']
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
# More than one neighborhood can exist in one postal code area.
df = df.groupby(['PostalCode', 'Borough']).agg(', '.join)
df = df.reset_index()
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [17]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighborhood']=='Not assigned']
# no such cell

,PostalCode,Borough,Neighborhood


In [18]:
df.shape

(103, 3)

## What I have done so far on the data cleaning above?

1. Import and scrap wikipage using BeautifulSoup package.
2. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
3. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
4. The rows of same PostalCode and Borough will be combined into one row with the neighborhoods separated with a comma.
5. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.